In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from delta import configure_spark_with_delta_pip
import os
from delta import *

In [2]:
SPARK_MASTER = os.environ['SPARK_MASTER']

In [3]:
print(SPARK_MASTER)

spark://spark-master:7077


In [4]:
# Create a SparkConf object and override a specific setting
conf = SparkConf() \
    .setMaster("spark://spark-master:7077") \
    .setAppName("MyApp") \
    .set("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .set("spark.hadoop.fs.s3a.access.key", "minio") \
    .set("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", False) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.jars.packages","io.delta:delta-spark_2.12:3.2.0,org.apache.hadoop:hadoop-aws:3.3.6")\
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [5]:
# Create a SparkSession with the configured SparkConf
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyteruser/.ivy2/cache
The jars for the packages stored in: /home/jupyteruser/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-69775c52-8b57-407f-bf2e-29aec8e9fd44;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.6 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.367 in central
	found org.wildfly.openssl#wildfly-openssl;1.1.3.Final in central
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.367/aws-java-sdk-bundle-1.12.367.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.367!aws-java-sdk-bundle.jar (7879ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.1.3.Final/wildfly-openssl-1.1.3.F

In [6]:
print(spark.conf.get("spark.app.name")) 

MyApp


In [7]:
# Verify Delta Lake is enabled
print(spark.conf.get("spark.sql.extensions"))

io.delta.sql.DeltaSparkSessionExtension


In [8]:
# Sample DataFrame
data = [("John", 30), ("Anna", 25), ("Mike", 40)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

In [9]:
# Define the path in MinIO where you will store the Delta table
# delta_table_path = "s3a://test/delta-table-2"
delta_table_path = "/tmp/delta/table"

In [10]:
# Write DataFrame as Delta format to MinIO (S3 bucket)
df.write.format("delta").mode("overwrite").save(delta_table_path)
print(f"Delta table written to {delta_table_path}")

24/09/20 09:48:41 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 2) (172.19.0.5 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/tmp/delta/table/_delta_log/00000000000000000000.json does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:781)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.

Py4JJavaError: An error occurred while calling o68.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 5) (172.19.0.5 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/tmp/delta/table/_delta_log/00000000000000000000.json does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:781)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasou

In [ ]:
# delta_df = spark.read.format("delta").load(delta_table_path)

In [ ]:
# delta_df.show()

# 